In [ ]:
#| default_exp hwc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastcore.utils import *
import pandas as pd
from cylinder.demand import load_demand
from cylinder.power import load_power
import matplotlib.pyplot as plt
import random
from collections import deque
import numpy as np

Establish a basic class of hot water cylinder that can be used to model a mixed hot water cylinder. The model is based on the following assumptions:

$C_v\rho V \frac{dT(t)}{dt} = C_v\rho F(t)[T_{in} - T(t)]+Q(t)-UA\text{[T(t)-}T_{atm}]$

In [ ]:
#| export
class HWC():
  """
  Simple mixed model for a hot water cylinder
  """
  # Define constants for clearer code
  def __init__(self,
              element = 3000, # Watts
              T_set = 65, # °C
              T_deadband = 2, # °C
              radius =.25,  # Meters
              height = 1, # Meters
              unit=None,  # Unit of measure for volume
              bedrooms=None,
              id = None):
    super(HWC, self).__init__()

    self.T_ambient = 15 #  Air temperature that the cylinder located in °C
    self.T_cold = 15 # Make up water temperature °C
    self.T_demand = 45 #T_demand - temperature of the end use (shower)  °C
    self.T_deadband = T_deadband #T_deadband  - thermostat deadband °C
    self.T_set = T_set #T_set - thermostat set point °C
    self.U = 0.5/60/1000 # heat transfer coefficient kJ/min m2K
    self.Cv = 4.184 
    self.rho = 1000 
    self.Area = 2*np.pi*radius*(radius+height)
    self.Volume = np.pi*radius**2*height
    self.Q = element/1000
    self.thermostat = False
    self.bedrooms = bedrooms
    self.id = id
    self.reset()

Provide a mechanisim to set the thermostat state based on the model temperature

In [ ]:
#| export
@patch
def _thermostat(self:HWC):
    if (self.T < self.T_set- self.T_deadband) :
        self.thermostat = 1
    elif (self.T > self.T_set) :
        self.thermostat = 0
    return

Update the model temperature based on the thermostat state / flow / heat loss

In [ ]:
#| export
@patch
def _update_temperatures(self:HWC, action = 1, ts = 60):
    '''
    Use the model from M Jack Paper to update the hwc temperature
    Takes existing state (temperature)
    '''
    temperature = self.T # existing temperatue
    temperature += self.flow/self.Volume*(self.T_cold-self.T) * ts # change in temperature due to flow mixing
    temperature += action * self.thermostat *  self.Q / (self.Cv*self.rho*self.Volume) * ts # change in temperature due to element
    temperature += self.U * self.Area * (self.T_ambient-self.T) * ts # change in temperature due to heat loss
    return temperature

Provide some initialisation functions to set the model up with some default values

In [ ]:
#| export
@patch
def reset(self:HWC):
    self.T = self.T_set + np.random.uniform(-4, 0)
    self.thermostat = 0
    multiplier = 2 if self.bedrooms is None else self.bedrooms
    self.thermogram = np.ones([7,24])*(.5+multiplier*.1)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()